In [ ]:
import read_lif
fpath ='/group/jug/ashesh/data/HHMI25_v2/12_noon_female_con_liv4.lif'
reader = read_lif.Reader(fpath)
# reader = read_lif.Reader('/group/jug/ashesh/data/HHMI_v2/6am_am_female_LIv4.lif')
series = reader.getSeries()

chosen = series[0]  # choose first image in the lif file


In [ ]:
len(series)

In [ ]:
import numpy as np

image0 = chosen.getFrame(T=0, channel=0, dtype = np.uint16)
image1 = chosen.getFrame(T=0, channel=1, dtype = np.uint16)
image2 = chosen.getFrame(T=0, channel=2, dtype = np.uint16)
image3 = chosen.getFrame(T=0, channel=3, dtype = np.uint16)
image4 = chosen.getFrame(T=0, channel=4, dtype = np.uint16)
image5 = chosen.getFrame(T=0, channel=5, dtype = np.uint16)
data = np.stack((image0, image1, image2, image3, image4, image5))
print(image0.shape, image1.shape, image2.shape, image3.shape, image4.shape, image5.shape)

In [ ]:
print(image0.shape, image1.shape, image2.shape, image3.shape, image4.shape, image5.shape)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from disentangle.analysis.plot_utils import clean_ax
_,ax = plt.subplots(figsize=(18,3),ncols=6)
ax[0].imshow(image0[20,::4,::4], cmap='gray')
ax[1].imshow(image1[20,::4,::4], cmap='gray')
ax[2].imshow(image2[30,::4,::4], cmap='gray')
ax[3].imshow(image3[20,::4,::4], cmap='gray')
ax[4].imshow(image4[20,::4,::4], cmap='gray')
ax[5].imshow(image5[20,::4,::4], cmap='gray')
clean_ax(ax)
# reduce the space between subplots
plt.subplots_adjust(wspace=0.01)
# pd.Series(image3.reshape(-1)).value_counts().sort_index().iloc[3:].plot(marker='.')

In [ ]:
data.shape

In [ ]:
def get_noisy_data(base_data, sigma, poisson):
    gaus_noise = 0
    pois_data = base_data
    if sigma is not None:
        gaus_noise = np.random.normal(0, sigma, base_data.shape)

    if poisson is not None:
        pois_data = np.random.poisson(base_data // poisson) * poisson

    noisy_data = pois_data + gaus_noise
    noisy_data [ noisy_data < 0 ] = 0
    noisy_data[ noisy_data > 65535 ] = 65535
    return noisy_data.astype(np.uint16)

base_data = image2[:4]
sigma = 4000 
poisson = 10000
noisy_data = get_noisy_data(base_data, sigma, poisson)

In [ ]:
hs = 400
ws = 400
sz = 100
img_idx = 0
_,ax = plt.subplots(figsize=(8,4),ncols=2)
ax[0].imshow(base_data[img_idx, hs:hs+sz,ws:ws+sz], cmap='magma')
ax[1].imshow(noisy_data[img_idx, hs:hs+sz,ws:ws+sz], cmap='magma')

In [ ]:
noisy_data = data.copy()
for i in range(6):
    noisy_data[i] = get_noisy_data(noisy_data[i], sigma, poisson)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sz = 400
from disentangle.analysis.plot_utils import clean_ax
_,ax = plt.subplots(figsize=(18,6),ncols=6,nrows=2)
ax[0,0].imshow(data[0][10,:sz, :sz], cmap='magma')
ax[0,1].imshow(data[1][10,:sz, :sz], cmap='magma')
ax[0,2].imshow(data[2][10,:sz, :sz], cmap='magma')
ax[0,3].imshow(data[3][10,:sz, :sz], cmap='magma')
ax[0,4].imshow(data[4][10,:sz, :sz], cmap='magma')
ax[0,5].imshow(data[5][10,:sz, :sz], cmap='magma')

ax[1,0].imshow(noisy_data[0][10,:sz, :sz], cmap='magma')
ax[1,1].imshow(noisy_data[1][10,:sz, :sz], cmap='magma')
ax[1,2].imshow(noisy_data[2][10,:sz, :sz], cmap='magma')
ax[1,3].imshow(noisy_data[3][10,:sz, :sz], cmap='magma')
ax[1,4].imshow(noisy_data[4][10,:sz, :sz], cmap='magma')
ax[1,5].imshow(noisy_data[5][10,:sz, :sz], cmap='magma')

clean_ax(ax)
# reduce the space between subplots
plt.subplots_adjust(wspace=0.01)
# pd.Series(image3.reshape(-1)).value_counts().sort_index().iloc[3:].plot(marker='.')

In [ ]:
import os
outputdir = f'/group/jug/ashesh/data/HHMI25_v2_noisy/gaus-{sigma}_pois-{poisson}'
os.makedirs(outputdir, exist_ok=True)
out_fpath = os.path.join(outputdir, os.path.basename(fpath))
print(out_fpath)
